In [1]:
from bs4 import BeautifulSoup
import urllib
import request
import urllib.request
import requests
import csv

In [2]:
def innerHTML(element):
    return element.decode_contents(formatter="html")

def get_url(body):
    return body.get('data-href')

def get_name(body):
    return body.find('span', {'class':'jcn'}).a.get('title')

def which_digit(html):
    mappingDict={'icon-ji':9,
                'icon-dc':'+',
                'icon-fe':'(',
                'icon-hg':')',
                'icon-ba':'-',
                'icon-lk':8,
                'icon-nm':7,
                'icon-po':6,
                'icon-rq':5,
                'icon-ts':4,
                'icon-vu':3,
                'icon-wx':2,
                'icon-yz':1,
                'icon-acb':0,
                }
    return mappingDict.get(html,'')

In [3]:
def get_phone_number(body):
    i=0
    phoneNo = "No Number!"
    try:
            
        for item in body.find('p',{'class':'contact-info'}):
            i+=1
            if(i==2):
                phoneNo=' '
                try:
                    for element in item.find_all(class_=True):
                        classes = []
                        classes.extend(element["class"])
                        phoneNo+=str((which_digit(classes[1])))
                except:
                    pass
    except:
        pass
    body = body['data-href']
    soup = BeautifulSoup(body, 'html.parser')
    for a in soup.find_all('a', {"id":"whatsapptriggeer"} ):
        # print (a)
        phoneNo = str(a['href'][-10:])


    return str(phoneNo)

In [4]:
def get_rating(body):
    rating = 0.0
    text = body.find('span', {'class':'star_m'})
    if text is not None:
        for item in text:
            rating += float(item['class'][0][1:])/10

    return rating

def get_rating_count(body):
    text = body.find('span', {'class':'rt_count'}).string

    # Get only digits
    rating_count =''.join(i for i in text if i.isdigit())
    return rating_count

def get_address(body):
    return body.find('span', {'class':'mrehover'}).text.strip()

def get_location(body):
    text = body.find('a', {'class':'rsmap'})
    if text == None:
        return
    text_list = text['onclick'].split(",")
    
    latitutde = text_list[3].strip().replace("'", "")
    longitude = text_list[4].strip().replace("'", "")
    
    return latitutde + ", " + longitude

In [6]:
page_number = 1
service_count = 1


fields = ['Name', 'Phone', 'Rating', 'Rating Count', 'Address', 'Location', 'Url']
out_file = open('Scrup.csv','w')
csvwriter = csv.DictWriter(out_file, delimiter=',', fieldnames=fields)

# Write fields first
#csvwriter.writerow(dict((fn,fn) for fn in fields))

while True:
    # Check if reached end of result
    if page_number > 2:
        break

    url="https://www.justdial.com/Delhi/Diagnostic-Centres/nct-10159446/page-%s" % (page_number)
    print(url)
    req = urllib.request.Request(url, headers={'User-Agent' : "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"}) 
    page = urllib.request.urlopen( req )
    # page=urllib2.urlopen(url)

    soup = BeautifulSoup(page.read(), "html.parser")
    services = soup.find_all('li', {'class': 'cntanr'})

    # Iterate through the 10 results in the page
    for service_html in services:

        # Parse HTML to fetch data     
        dict_service = {}
        name = get_name(service_html)
        print(name);
        url = get_url(service_html)
        phone = get_phone_number(service_html)
        rating = get_rating(service_html)
        count = get_rating_count(service_html)
        address = get_address(service_html)
        location = get_location(service_html)
        if name != None:
            dict_service['Name'] = str(name)
        if url != None:
            dict_service['Url'] = str(url)    
        if phone != None:
            print('getting phone number')
            dict_service['Phone'] = str(phone)
        if rating != None:
            dict_service['Rating'] = rating
        if count != None:
            dict_service['Rating Count'] = count
        if address != None:
            dict_service['Address'] = address
        if location != None:
            dict_service['Location'] = location

        # Write row to CSV
        csvwriter.writerow(dict_service)
        print("#" + str(service_count) + " " , dict_service)
        service_count += 1

    page_number += 1

out_file.close()

https://www.justdial.com/Delhi/Diagnostic-Centres/nct-10159446/page-1
Orbit Imaging & Pathlab Ltd in Pusa Road, Delhi
getting phone number
#1  {'Name': 'Orbit Imaging & Pathlab Ltd in Pusa Road, Delhi', 'Url': 'https://www.justdial.com/Delhi/Orbit-Imaging-Pathlab-Ltd-Opposite-Karol-Bagh-Metro-Station-Pillar-Number-124-Pusa-Road/011PXX11-XX11-110921092640-Z4C2_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz', 'Phone': ' +(91)-9152438155', 'Rating': 3.9, 'Rating Count': '919', 'Address': '35 B, Pusa Road, Delhi - 110005, Opposite Karol Bagh Metro Station & Pillar Number 124'}
One Plus Ultrasound Lab in Pitampura, Delhi
getting phone number
#2  {'Name': 'One Plus Ultrasound Lab in Pitampura, Delhi', 'Url': 'https://www.justdial.com/Delhi/One-Plus-Ultrasound-Lab-Near-Harsh-Vihar-Pitampura/011PXX11-XX11-150405133035-L3D3_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz', 'Phone': ' +(91)-9152392226', 'Rating': 4.3, 'Rating Count': '143', 'Address': '47, Pitampura, Delhi - 110034, Near Harsh Vihar'}
Dr

C:\Users\admin\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www.justdial.com/Delhi/Krishna-Diagnostics-Dr-Vandanas-3d-4d-Ultrasound-Lab-Near-D-MALL-Niti-Khand-2-indirapuram/011PXX11-XX11-091206230605-B3M6_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\admin\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www.justdial.com/Delhi/Deewan-Diagnostics-Opposite-New-Bus-Stand-and-Near-Hotel-Pelican-Patel-Nagar-2-Patel-Nagar-2/011PXX11-XX11-100115162332-Y2Q9_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document 

Dr. Dangs Lab Llp in Safdarjung Development Area, Delhi
getting phone number
#11  {'Name': 'Dr. Dangs Lab Llp in Safdarjung Development Area, Delhi', 'Url': 'https://www.justdial.com/Delhi/Dr-Dangs-Lab-Llp-Near-Aurobindo-Market-Safdarjung-Development-Area/011PXX11-XX11-121111101818-E1K2_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz', 'Phone': ' +(91)-9152504245', 'Rating': 4.5, 'Rating Count': '3038', 'Address': 'C 2/1, Safdarjung Development Area, Delhi - 110016, Near Aurobindo Market'}
Unipath Diagnostics in Lajpat Nagar 2, Delhi
getting phone number
#12  {'Name': 'Unipath Diagnostics in Lajpat Nagar 2, Delhi', 'Url': 'https://www.justdial.com/Delhi/Unipath-Diagnostics-Near-Rampul-Lajpat-Nagar-2/011PXX11-XX11-100122125252-V9T9_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz', 'Phone': ' +(91)-9152902271', 'Rating': 4.3, 'Rating Count': '196', 'Address': 'G 49, Lajpat Nagar 2, Delhi - 110024, Near Rampul'}
Kataria Healthcare in Greater Kailash 4, Delhi
getting phone number
#13  {'Name': 'Kata

C:\Users\admin\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www.justdial.com/Delhi/Unipath-Diagnostics-Near-Rampul-Lajpat-Nagar-2/011PXX11-XX11-100122125252-V9T9_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\admin\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www.justdial.com/Delhi/City-Diagnostic-And-Ct-Scan-Centre-Near-SBI-BANK-Khandsa-Road/011P1230703342Y7P3K9_BZDET?xid=RGVsaGkgRGlhZ25vc3RpYyBDZW50cmVz" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\admin\Anaconda3\lib\site-packages\bs4\_